In [ ]:
%%html

<style>

.MathJax { font-size: 1.1em; }
.tableplain { vertical-align: top; }

.red { color: red; }
.green { color: green; }
.blue { color: blue; }
.black { color: black; }

dl { display: block; margin: 1em 0 1em; }
dt { display: block; margin-left: 20px; }
dd { display: block; margin-left: 40px; }

</style>

<span style='font-size: 300%; font-weight: bold;'>
<a style="position: fixed; bottom:5%; right:5%;" href="#top">&#8679;</a>
</span>

# Testing precision and compression of Double32_t and Float16_t

ROOT provides the Double32_t and Float16_t types to control precision of
real numbers when saved on disk. There is an ongoing discussion about using the
ROOT's `Float16_t` type in place of `Short_t` for data members in `picoDst`.
`Float16_t` can be used to pack real numbers in a few different ways
[TBufferFile::WriteFloat16()](https://root.cern.ch/doc/master/classTBufferFile.html#a61098f97baa261b6a501fbf5c09aba62)

A couple of statements which we would like to verify here have been made during the
discussion:

 - `Float16_t` provides a better precision than `Short_t` when limits are specified
 - `Float16_t` provides better compression than `Short_t`

In the [Summary and Conclusions](#Summary-and-Conclusions) we show that our findings
based on this study contradict the above statements.

We start with the code found in ROOT tutorials that compares different packing modes
of `Double32_t` [double32.C](https://root.cern.ch/doc/master/double32_8C.html).
Both `Double32_t` and `Float16_t` implement the same idea of controlled packing of
the standard representation of single and double precision real numbers into smaller
size formats.

The original code was modified by adding various packing modes for `Float16_t`, `Short_t`,
`UShort_t`, and `UInt_t`. Here is the list of defined variables going to corresponging
branches in the output TTree:


```c++
struct DemoDouble32
{
   Double_t    fD64;     //reference member with full double precision
   Double32_t  fF32;     //saved as a 32 bit Float_t

   Double32_t  fI32;     //[-100,100]    saved as a 32 bit unsigned int
   Double32_t  fI30;     //[-100,100,30] saved as a 30 bit unsigned int
   Double32_t  fI28;     //[-100,100,28] saved as a 28 bit unsigned int
   Double32_t  fI26;     //[-100,100,26] saved as a 26 bit unsigned int
   Double32_t  fI24;     //[-100,100,24] saved as a 24 bit unsigned int
   Double32_t  fI22;     //[-100,100,22] saved as a 22 bit unsigned int
   Double32_t  fI20;     //[-100,100,20] saved as a 20 bit unsigned int
   Double32_t  fI18;     //[-100,100,18] saved as a 18 bit unsigned int
   Double32_t  fI16;     //[-100,100,16] saved as a 16 bit unsigned int
   Double32_t  fI14;     //[-100,100,14] saved as a 14 bit unsigned int
   Double32_t  fI12;     //[-100,100,12] saved as a 12 bit unsigned int
   Double32_t  fI10;     //[-100,100,10] saved as a 10 bit unsigned int
   Double32_t  fI8;      //[-100,100, 8] saved as a  8 bit unsigned int
   Double32_t  fI6;      //[-100,100, 6] saved as a  6 bit unsigned int
   Double32_t  fI4;      //[-100,100, 4] saved as a  4 bit unsigned int
   Double32_t  fI2;      //[-100,100, 2] saved as a  2 bit unsigned int

   Double32_t  fR16;     //[0,  0, 16] saved as a 32 bit float with a 16 bits mantissa
   Double32_t  fR14;     //[0,  0, 14] saved as a 32 bit float with a 14 bits mantissa
   Double32_t  fR12;     //[0,  0, 12] saved as a 32 bit float with a 12 bits mantissa
   Double32_t  fR10;     //[0,  0, 10] saved as a 32 bit float with a 10 bits mantissa
   Double32_t  fR8;      //[0,  0,  8] saved as a 32 bit float with a  8 bits mantissa
   Double32_t  fR6;      //[0,  0,  6] saved as a 32 bit float with a  6 bits mantissa
   Double32_t  fR4;      //[0,  0,  4] saved as a 32 bit float with a  4 bits mantissa
   Double32_t  fR2;      //[0,  0,  2] saved as a 32 bit float with a  2 bits mantissa

   Float16_t   fI16F;    //[-100, 100, 16]
   Float16_t   fI14F;    //[-100, 100, 14]
   Float16_t   fI12F;    //[-100, 100, 12]
   Float16_t   fI10F;    //[-100, 100, 10]
   Float16_t   fI8F;     //[-100, 100,  8]
   Float16_t   fI6F;     //[-100, 100,  6]
   Float16_t   fI4F;     //[-100, 100,  4]
   Float16_t   fI2F;     //[-100, 100,  2]

   Float16_t   fR16F;    //[0, 0, 16]
   Float16_t   fR14F;    //[0, 0, 14]
   Float16_t   fR12F;    //[0, 0, 12]
   Float16_t   fR10F;    //[0, 0, 10]
   Float16_t   fR8F;     //[0, 0,  8]
   Float16_t   fR6F;     //[0, 0,  6]
   Float16_t   fR4F;     //[0, 0,  4]
   Float16_t   fR2F;     //[0, 0,  2]

   Short_t     fSh;
   UShort_t    fUSh;
   UInt_t      fUI;
}
```

The modified code (double32.C) is used for all tests performed in this study, and can be
found in this repository https://github.com/plexoos/my-tests/tree/master/root-branch-compress.

Several distributions are considered for this test:

 - Uniform [-100, 100]
 - Uniform with 5% overflow [-105, 105]
 - Normal distributions with sigma = 10, 15, 20, 25, 30, 35, 40

In [ ]:
import ROOT

ROOT.enableJSVis()
ROOT.gStyle.SetPadRightMargin(0.30)
ROOT.gStyle.SetOptStat("emroui")
ROOT.gStyle.SetStatW(0.28)
ROOT.gStyle.SetStatH(0.50)
ROOT.gStyle.SetStatX(1)
ROOT.gStyle.SetStatY(1)
ROOT.gStyle.SetLabelSize(0.08, "XYZ")
ROOT.gStyle.SetTitleFontSize(0.10)

# Uniform distribution

Let's compare the types for a uniformly distributed values in the range [-100, 100]

The first row shows the original unpacked values

In [ ]:
test_file = ROOT.TFile("DemoDouble32_flat_nOF.root")
test_tree = test_file.Get("T")

canvas = ROOT.TCanvas("canvas_multi_1", "canvas", 1000, 1500)
canvas.Divide(2, 9)

branch_names = ["fD64", "fF32",
                "fI16", "fR16",
                "fI14", "fR14",
                "fI12", "fR12",
                "fI10", "fR10",
                "fI8", "fR8",
                "fI6", "fR6",
                "fI4", "fR4",
                "fI2", "fR2"]

i_pad = 1

for branch_name in branch_names:
    canvas.cd(i_pad)
    test_tree.Draw(branch_name)
    i_pad += 1

canvas.Draw()

## Comparison of precisions offered by different packing modes

In [ ]:
saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

There are three points for each type with the following meaning

<dl>
    
<dt class="black">black</dt>
<dd>
Represent the precision of the packed value $x_t$ relative to the original value $x$
after it was written and read back

$$ -\log_{10} \Big( \frac{x - x_t}{x} \Big) $$

For example, the relative precision of 2 means that most of the time $x_t$ will be within 1%
of the original value $x$
    
</dd>
 
<dt class="red">red</dt> 
<dd>
Similar to the black points but defined as
    
$$ -\log_{10} \Big( \frac{x - x_t}{xmax - xmin} \Big) $$

This quantity significantly degrades when the specified range is not optimal, e.g. when the
variable does not fit within the range causing overflows and underflows
</dd>

<dt class="blue">blue</dt> 
<dd>
The compression factor is self explanatory, the higher the better
</dd>

</dl>

# Normal distribution

Now let's look at a normally distributed variable with sigma = 20

The first row shows the original unpacked values

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s20.root")
test_tree = test_file.Get("T")

canvas = ROOT.TCanvas("canvas_multi_2", "canvas", 1000, 1600)
canvas.Divide(2, 10)

branch_names = ["fD64", "fF32",
                "fI16", "fR16",
                "fI14", "fR14",
                "fI12", "fR12",
                "fI10", "fR10",
                 "fI8",  "fR8",
                 "fI6",  "fR6",
                 "fI4",  "fR4",
                 "fI2",  "fR2",
                "fUSh", "fUI"]

i_pad = 1

for branch_name in branch_names:
    canvas.cd(i_pad)
    test_tree.Draw(branch_name)
    i_pad += 1

canvas.Draw()

## Comparison of precisions offered by different packing modes

In [ ]:
saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s10.root")
test_tree = test_file.Get("T")

saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s15.root")
test_tree = test_file.Get("T")

saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s20.root")
test_tree = test_file.Get("T")

saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

Note that with the width greater than 25 the values do not completely fit
into the [-100. 100] range

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s25.root")
test_tree = test_file.Get("T")

saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s30.root")
test_tree = test_file.Get("T")

saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s35.root")
test_tree = test_file.Get("T")

saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s40.root")
test_tree = test_file.Get("T")

saved_canvas = test_file.Get("c1")
saved_canvas.Draw()

In [ ]:
canvas = ROOT.TCanvas("canvas_multi_3", "canvas", 1000, 1600)
canvas.Divide(2, 10)

branch_names = ["fD64", "fF32",
                "fI16", "fR16",
                "fI14", "fR14",
                "fI12", "fR12",
                "fI10", "fR10",
                 "fI8",  "fR8",
                 "fI6",  "fR6",
                 "fI4",  "fR4",
                 "fI2",  "fR2",
                "fUSh",  "fUI"]

i_pad = 1

for branch_name in branch_names:
    canvas.cd(i_pad)
    test_tree.Draw(branch_name)
    i_pad += 1

canvas.Draw()

## Relative error distributions

Here we take a more detailed look at the gaussian distribution packed as a floating
point number with truncated mantissa and as a 32-bit integer. We plot the
packed/unpacked variable, the relative error, and the relative errorr with respect
to the range.

The distributions of relative precisions very well agree with the expected machine
accuracy for the requested number of bits in the significand (//[0, 0, nbits]).

$$ \Big| \frac{x - x_t}{x} \Big| \le 2^{-(\text{nbits}+1)}$$

Note
that the actual number of bits is nbits + 1 because there is sign bit. The case
//[0, 0, 16] seems to be special in that regard.


### Packed as floating point

In [ ]:
test_file = ROOT.TFile("DemoDouble32_gauss_s20.root")
test_tree = test_file.Get("T")

canvas = ROOT.TCanvas("canvas_multi_detailed", "canvas", 1500, 2000)
canvas.Divide(3, 9)

branch_names = [
    "fD64", "fF32", "",
    "fR16F", "(fD64 - fR16F)/fD64", "(fD64 - fR16F)/(fMax - fMin)",
    "fR14F", "(fD64 - fR14F)/fD64", "(fD64 - fR14F)/(fMax - fMin)",
    "fR12F", "(fD64 - fR12F)/fD64", "(fD64 - fR12F)/(fMax - fMin)",
    "fR10F", "(fD64 - fR10F)/fD64", "(fD64 - fR10F)/(fMax - fMin)",
    "fR8F",  "(fD64 - fR8F) /fD64", "(fD64 - fR8F) /(fMax - fMin)",
    "fR6F",  "(fD64 - fR6F) /fD64", "(fD64 - fR6F) /(fMax - fMin)",
    "fR4F",  "(fD64 - fR4F) /fD64", "(fD64 - fR4F) /(fMax - fMin)",
    "fR2F",  "(fD64 - fR2F) /fD64", "(fD64 - fR2F) /(fMax - fMin)"
]

i_pad = 1

for branch_name in branch_names:
    canvas.cd(i_pad)
    test_tree.Draw(branch_name)
    i_pad += 1

canvas.Draw()

### Packed as 32 bit integer

In [ ]:
branch_names = [
    "fD64", "", "",
    "fI16F", "(fD64 - fI16F)/fD64", "(fD64 - fI16F)/(fMax - fMin)",
    "fI14F", "(fD64 - fI14F)/fD64", "(fD64 - fI14F)/(fMax - fMin)",
    "fI12F", "(fD64 - fI12F)/fD64", "(fD64 - fI12F)/(fMax - fMin)",
    "fI10F", "(fD64 - fI10F)/fD64", "(fD64 - fI10F)/(fMax - fMin)",
    "fI8F",  "(fD64 - fI8F) /fD64", "(fD64 - fI8F) /(fMax - fMin)",
    "fI6F",  "(fD64 - fI6F) /fD64", "(fD64 - fI6F) /(fMax - fMin)",
    "fI4F",  "(fD64 - fI4F) /fD64", "(fD64 - fI4F) /(fMax - fMin)",
    "fI2F",  "(fD64 - fI2F) /fD64", "(fD64 - fI2F) /(fMax - fMin)"
]

i_pad = 1

for branch_name in branch_names:
    canvas.cd(i_pad)
    test_tree.Draw(branch_name)
    i_pad += 1

canvas.Draw()

### Bin width for values with truncated mantissa

In order to avoid weird looking distributions one should choose the bin
width to be larger than the absolute precision at the range maximum

In [ ]:
n_pads = 7

canvas = ROOT.TCanvas("canvas_multi_bins", "canvas", 800, 200*n_pads)
canvas.Divide(1, n_pads)

# Print out precision for fR8F at different values
for value in range(10, 100, 10):
    abs_precision = value * 2**-9
    print('fR8F abs. precision @ {}: {}' .format(value, abs_precision))

start_n_bins = 100

for i_pad in range(1, n_pads+1):
    canvas.cd(i_pad)
    n_bins = start_n_bins if i_pad == 1 else start_n_bins * 2 
    
    # Print out the bin width
    print('fR8F>>h{}({}, -100, 100) ==> bin width: {}'.format(i_pad, n_bins, 200/n_bins))
    
    test_tree.Draw( 'fR8F>>h{}({}, -100, 100)'.format(i_pad, n_bins) )
    start_n_bins = n_bins
    i_pad += 1

canvas.Draw()

# Summary and Conclusions

We compared different packing modes of the `Float16_t` type in ROOT for uniformly and
normally distributed variables. In the comparison plots one should focus on the bins
labeled `fSh`, `fUSh`, `fI16F`, and `fR16F` and comapre relative precision and
compression factors for the corresponding types.

The following conclusions can be made:

 - `Float16_t` is packed into a 32-bit integer using `nbits` when //[min, max, nbits] format
 is specified. If `nbits` = 16 this packing does not offer any better precision than
 when `UShort_t` is used as an underlying type
   
 - `Float16_t` is packed into a 16-bit mantissa + 8-bit exponent when the range is not specified,
 i.e. //[0, 0, nbits]. In this case the relative precision can be better than with `UShort_t`.
 Note that this is not the currently proposed change
 
 - In both modes `Float16_t` does not compress as good as `UShort_t` even with the maximum
 compression in ROOT5

 - The packing mode of `Float16_t` that closely matches `UShort_t` in compression and provides
 a better precision is defined as //[0, 0, 8]